# 預測航班是否會因為天氣因素而延遲&延遲多久

## Step1: Load weather data 、查看資料類型和分布

* load weather data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import pylab as pl #調整圖的x軸標籤顯示角度 #pl.xticks(rotation=90)
%matplotlib inline

In [2]:
weather_2 = pd.read_csv(r"C:\Users\USER\Desktop\w2.csv")

C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8,10,11,14,15,20,23,25,39,40,59,67,68,69,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


* 查看資料量

In [3]:
print(weather_2.shape) #天氣1資料共66340筆，90個欄位
total2_len = weather_2.shape[0]

(66340, 90)


## EDA

1. 擷取出2013年9月資料

2. 挑選出可能會影響航班的天氣特徵

3. 將很長的STATION_NAME替換成較短的IATA機場代碼

4. 欄位資料轉換(應該要是數值型，卻不是數值型的欄位)

5. 欄位處理(將日期時間分開成兩個獨立欄位、新增一欄"月日"欄位)

6. 刪除分鐘數不為53的列

7. 填補HOURLYPrecip的空值，用0取代

8. 特徵組合:新增"蒸發量"欄位(乾球溫度與濕球溫度差距越大，表示相對溼度越小)，刪除乾濕球溫度欄位

In [4]:
## 1. 擷取出weather_2 2013年9月資料

# 將日期時間獨立成兩個欄位
weather_2["YMD"] = weather_2["DATE"].apply(lambda x : x.split(" ")[0])
weather_2["TIME"] = weather_2["DATE"].apply(lambda x : x.split(" ")[1])

# 刪除不是"2013-09"的列
weather_2["temp"] = weather_2["YMD"].apply(lambda x : x[:7]) # 暫放年月
not_sep_2 = weather_2[weather_2["temp"] != "2013-09"] # 133414筆
weather_2 = weather_2.drop(not_sep_2.index) # 刪除不是2013年9月的資料

# 將"temp"欄位刪除
weather_2.drop(["temp"],axis=1,inplace=True)

print(weather_2.shape) #刪完後剩下的資料共有1683筆，92個欄位

(1683, 92)


In [5]:
## 2. 由於總欄位數過多，因此直接挑選可能會對航班有影響的天氣欄位

weather_2 = weather_2[["STATION_NAME","HOURLYVISIBILITY","HOURLYDRYBULBTEMPC","HOURLYWETBULBTEMPC"
                       ,"HOURLYDewPointTempC","HOURLYRelativeHumidity","HOURLYWindSpeed","HOURLYPrecip","YMD","TIME"]].copy()

* 刪除重複資料(列)

In [6]:
weather_2.drop_duplicates()

,STATION_NAME,HOURLYVISIBILITY,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPC,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYPrecip,YMD,TIME
0,HOUSTON WILLIAM P HOBBY AIRPORT TX US,10.00,25,22.6,21.7,82.0,3.0,0.00,2013-09-01,00:53
1,HOUSTON WILLIAM P HOBBY AIRPORT TX US,10.00,25,22.6,21.7,82.0,6.0,0.00,2013-09-01,01:53
2,HOUSTON WILLIAM P HOBBY AIRPORT TX US,10.00,24.4,22.4,21.7,85.0,3.0,0.00,2013-09-01,02:53
3,HOUSTON WILLIAM P HOBBY AIRPORT TX US,10.00,23.9,22.2,21.7,88.0,3.0,0.00,2013-09-01,03:53
4,HOUSTON WILLIAM P HOBBY AIRPORT TX US,10.00,23.9,22.2,21.7,88.0,0.0,0.00,2013-09-01,04:53
...,...,...,...,...,...,...,...,...,...,...
34296,DALLAS FAA AIRPORT TX US,10.00,25,20.1,16.1,58.0,6.0,0.00,2013-09-30,20:53
34297,DALLAS FAA AIRPORT TX US,10.00,24.4,19.9,16.7,62.0,7.0,0.00,2013-09-30,21:53
34298,DALLAS FAA AIRPORT TX US,10.00,24.4,19.9,16.7,62.0,7.0,0.00,2013-09-30,22:53
34299,DALLAS FAA AIRPORT TX US,10.00,23.9,19.8,17.2,66.0,7.0,0.00,2013-09-30,23:53


* 查看資料類型

In [7]:
weather_2.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1683 entries, 0 to 34300
Data columns (total 10 columns):
STATION_NAME              1683 non-null object
HOURLYVISIBILITY          1622 non-null object
HOURLYDRYBULBTEMPC        1623 non-null object
HOURLYWETBULBTEMPC        1622 non-null float64
HOURLYDewPointTempC       1623 non-null object
HOURLYRelativeHumidity    1623 non-null float64
HOURLYWindSpeed           1623 non-null float64
HOURLYPrecip              1509 non-null object
YMD                       1683 non-null object
TIME                      1683 non-null object
dtypes: float64(3), object(7)
memory usage: 144.6+ KB


* 欄位處理

In [8]:
# 3. STATION_NAME對映

weather_2['STATION_NAME'].replace('DALLAS FAA AIRPORT TX US','DAL',inplace=True)
weather_2['STATION_NAME'].replace('HOUSTON WILLIAM P HOBBY AIRPORT TX US','HOU',inplace=True)
weather_2['STATION_NAME'].replace('NASHVILLE INTERNATIONAL AIRPORT TN US','BNA',inplace=True)
weather_2['STATION_NAME'].replace('ST LOUIS LAMBERT INTERNATIONAL AIRPORT MO US','STL',inplace=True)

In [9]:
# 4. 錯誤數據(應該要是數值型，卻不是數值型的欄位)

def tryconvert(x):
        try:
            if str(x)[-1].isalpha():
                return(float(str(x)[:-1])) #若有夾雜英文字符，直接捨去字符部分
            else:
                return(float(str(x)))
        except:
            return(np.nan)

weather_2['HOURLYVISIBILITY'] = weather_2['HOURLYVISIBILITY'].apply(lambda x: tryconvert(x))
weather_2['HOURLYDRYBULBTEMPC'] = weather_2['HOURLYDRYBULBTEMPC'].apply(lambda x: tryconvert(x))
weather_2["HOURLYDewPointTempC"] = weather_2["HOURLYDewPointTempC"].apply(lambda x : tryconvert(x))
weather_2['HOURLYWindSpeed'] = weather_2['HOURLYWindSpeed'].apply(lambda x: tryconvert(x))
weather_2['HOURLYPrecip'] = weather_2['HOURLYPrecip'].apply(lambda x: tryconvert(x))


# 5. 將YMD的日期(月日)獨立成新欄位
# weather_2["Month"] = weather_2["YMD"].apply(lambda x : x.split("-")[1]) #暫存欄位
# weather_2["Day"] = weather_2["YMD"].apply(lambda x : x.split("-")[-1])  # 暫存欄位

# weather_2["Month_Day"] = weather_2["Month"].map(str)+weather_2["Day"].map(str)

# weather_2.drop(["Month","Day"],axis = 1,inplace=True) # 刪除暫存的欄位

# 5. 由於航班都是9月資料，因此將YMD的日獨立成新欄位
weather_2["Day"] = weather_2["YMD"].apply(lambda x : x.split("-")[-1])
weather_2.drop(["YMD"],axis = 1,inplace=True) 

In [10]:
# 6. 由於基本上w2測站天氣數據都是在每小時的53分紀錄，因此將不是53分的列刪除

# 新增"小時"欄位
weather_2["HOUR"] = weather_2["TIME"].apply(lambda x : int(x[:2]))

# 獨立"分"欄位，用來判斷此筆資料是否為53分的列
weather_2["min"] = weather_2["TIME"].apply(lambda x : int(x[-2:]))

# 將不是53分的資料抓出來
not_53 = weather_2[weather_2["min"] != 53]

# 刪除不是53分的列
weather_2 = weather_2.drop(not_53.index)

# 將weather_2["min"]欄位刪除
weather_2.drop(["min"],axis=1,inplace = True)

In [11]:
# 7. 由於HOURLYPrecip的值大多為0，因此用0取代空值

weather_2['HOURLYPrecip'].fillna(value=0,inplace=True)

In [12]:
# 8. 特徵組合:新增"蒸發量"欄位(乾球溫度與濕球溫度差距越大，表示蒸發量越大)

weather_2["evaporation"] = weather_2["HOURLYDRYBULBTEMPC"]-weather_2["HOURLYWETBULBTEMPC"]

#刪除乾溼球溫度欄位
weather_2.drop(["HOURLYDRYBULBTEMPC","HOURLYWETBULBTEMPC"],axis=1,inplace=True)

In [13]:
print("weather_2(DAL、HOU):",weather_2.shape)

weather_2(DAL、HOU): (1440, 10)


In [14]:
weather_2

,STATION_NAME,HOURLYVISIBILITY,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYPrecip,TIME,Day,HOUR,evaporation
0,HOU,10.0,21.7,82.0,3.0,0.0,00:53,01,0,2.4
1,HOU,10.0,21.7,82.0,6.0,0.0,01:53,01,1,2.4
2,HOU,10.0,21.7,85.0,3.0,0.0,02:53,01,2,2.0
3,HOU,10.0,21.7,88.0,3.0,0.0,03:53,01,3,1.7
4,HOU,10.0,21.7,88.0,0.0,0.0,04:53,01,4,1.7
...,...,...,...,...,...,...,...,...,...,...
34295,DAL,10.0,15.0,50.0,3.0,0.0,19:53,30,19,5.6
34296,DAL,10.0,16.1,58.0,6.0,0.0,20:53,30,20,4.9
34297,DAL,10.0,16.7,62.0,7.0,0.0,21:53,30,21,4.5
34298,DAL,10.0,16.7,62.0,7.0,0.0,22:53,30,22,4.5


In [15]:
# 將清整後的檔案匯出
weather_2.to_csv("C:/Users/USER/Desktop/weather2.csv",index=False)

In [16]:
# load weather data
weather_1 = pd.read_csv(r"C:\Users\USER\Desktop\w1.csv")

C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8,10,11,14,15,17,20,23,25,39,40,59,67,68,69,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


* 查看資料量

In [17]:
print(weather_1.shape) #天氣1資料共70799筆，90個欄位
total1_len = weather_1.shape[0]

(70799, 90)


In [18]:
weather_1.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70799 entries, 0 to 70798
Data columns (total 90 columns):
STATION                              70799 non-null object
STATION_NAME                         70799 non-null object
ELEVATION                            70799 non-null float64
LATITUDE                             70799 non-null float64
LONGITUDE                            70799 non-null float64
DATE                                 70799 non-null object
REPORTTPYE                           70799 non-null object
HOURLYSKYCONDITIONS                  62447 non-null object
HOURLYVISIBILITY                     57825 non-null object
HOURLYPRSENTWEATHERTYPE              13806 non-null object
HOURLYDRYBULBTEMPF                   68851 non-null object
HOURLYDRYBULBTEMPC                   68851 non-null object
HOURLYWETBULBTEMPF                   64564 non-null float64
HOURLYWETBULBTEMPC                   64564 non-null float64
HOURLYDewPointTempF                  68849 non-null object
H

## EDA

1. 擷取出2013年9月資料

2. 挑選出可能會影響航班的天氣特徵

3. 將很長的STATION_NAME替換成較短的IATA機場代碼

4. 欄位資料轉換(應該要是數值型，卻不是數值型的欄位)

5. 欄位處理(將日期時間分開成兩個獨立欄位、新增一欄"月日"欄位)

6. 刪除STL分鐘數不為51的列;刪除BNA分鐘數不為53的列

7. 處理空值(HOURLYPrecip-->用0取代 ; HOURLYWETBULBTEMPC-->用平均值取代)

8. 特徵組合:新增"蒸發量"欄位(乾球溫度與濕球溫度差距越大，表示相對溼度越小)，刪除乾濕球溫度欄位

In [19]:
## 1. 擷取出weather_1 2013年9月資料

# 將日期時間獨立成兩個欄位
weather_1["YMD"] = weather_1["DATE"].apply(lambda x : x.split(" ")[0])
weather_1["TIME"] = weather_1["DATE"].apply(lambda x : x.split(" ")[1])

# 刪除不是"2013-09"的列
weather_1["temp"] = weather_1["YMD"].apply(lambda x : x[:7]) # 暫放年月
not_sep_1 = weather_1[weather_1["temp"] != "2013-09"] # 68757筆
weather_1 = weather_1.drop(not_sep_1.index) # 刪除不是2013年9月的資料

# 將"temp"欄位刪除
weather_1.drop(["temp"],axis=1,inplace=True)

print(weather_1.shape) #刪完後剩下的資料共有2042筆，92個欄位

(2042, 92)


In [20]:
## 2. 由於總欄位數過多，因此直接挑選可能會對航班有影響的天氣欄位

weather_1 = weather_1[["STATION_NAME","HOURLYVISIBILITY","HOURLYDRYBULBTEMPC","HOURLYWETBULBTEMPC"
                       ,"HOURLYDewPointTempC","HOURLYRelativeHumidity","HOURLYWindSpeed","HOURLYPrecip","YMD","TIME"]].copy()

In [21]:
weather_1.drop_duplicates()

,STATION_NAME,HOURLYVISIBILITY,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPC,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYPrecip,YMD,TIME
0,ST LOUIS LAMBERT INTERNATIONAL AIRPORT MO US,10,27.8,24.5,23.3,77.0,6,NaN,2013-09-01,00:51
1,ST LOUIS LAMBERT INTERNATIONAL AIRPORT MO US,10,26.7,24.2,23.3,82.0,0,0.00,2013-09-01,01:51
2,ST LOUIS LAMBERT INTERNATIONAL AIRPORT MO US,10,25.6,23.8,23.3,87.0,5,0.00,2013-09-01,02:51
3,ST LOUIS LAMBERT INTERNATIONAL AIRPORT MO US,8,25,23.7,23.3,90.0,3,0.00,2013-09-01,03:51
4,ST LOUIS LAMBERT INTERNATIONAL AIRPORT MO US,9,25.6,23.8,23.3,87.0,0,0.00,2013-09-01,04:51
...,...,...,...,...,...,...,...,...,...,...
39468,NASHVILLE INTERNATIONAL AIRPORT TN US,NaN,18.9,16.4,15,78.0,6,NaN,2013-09-30,21:00
39469,NASHVILLE INTERNATIONAL AIRPORT TN US,10.00,18.9,16.2,14.4,75.0,0,0.00,2013-09-30,21:53
39470,NASHVILLE INTERNATIONAL AIRPORT TN US,10.00,18.9,16.4,15,78.0,0,0.00,2013-09-30,22:53
39471,NASHVILLE INTERNATIONAL AIRPORT TN US,10.00,18.9,16.4,15,78.0,3,0.00,2013-09-30,23:53


* 欄位處理

In [22]:
# 3. STATION_NAME對映

weather_1['STATION_NAME'].replace('DALLAS FAA AIRPORT TX US','DAL',inplace=True)
weather_1['STATION_NAME'].replace('HOUSTON WILLIAM P HOBBY AIRPORT TX US','HOU',inplace=True)
weather_1['STATION_NAME'].replace('NASHVILLE INTERNATIONAL AIRPORT TN US','BNA',inplace=True)
weather_1['STATION_NAME'].replace('ST LOUIS LAMBERT INTERNATIONAL AIRPORT MO US','STL',inplace=True)

In [23]:
# 4. 錯誤數據(應該要是數值型，卻不是數值型的欄位)

def tryconvert(x):
        try:
            if str(x)[-1].isalpha():
                return(float(str(x)[:-1])) #若有夾雜英文字符，直接捨去字符部分
            else:
                return(float(str(x)))
        except:
            return(np.nan)

weather_1['HOURLYVISIBILITY'] = weather_1['HOURLYVISIBILITY'].apply(lambda x: tryconvert(x))
weather_1['HOURLYDRYBULBTEMPC'] = weather_1['HOURLYDRYBULBTEMPC'].apply(lambda x: tryconvert(x))
weather_1["HOURLYDewPointTempC"] = weather_1["HOURLYDewPointTempC"].apply(lambda x : tryconvert(x))
weather_1['HOURLYWindSpeed'] = weather_1['HOURLYWindSpeed'].apply(lambda x: tryconvert(x))
weather_1['HOURLYPrecip'] = weather_1['HOURLYPrecip'].apply(lambda x: tryconvert(x))


# 5. 將YMD的日期(月日)獨立成新欄位
# weather_1["Month"] = weather_1["YMD"].apply(lambda x : x.split("-")[1]) #暫存欄位
# weather_1["Day"] = weather_1["YMD"].apply(lambda x : x.split("-")[-1])  # 暫存欄位

# weather_1["Month_Day"] = weather_1["Month"].map(str)+weather_1["Day"].map(str)

# weather_1.drop(["Month","Day"],axis = 1,inplace=True) # 刪除暫存的欄位

# 5. 由於航班都是9月資料，因此將YMD的日獨立成新欄位
weather_1["Day"] = weather_1["YMD"].apply(lambda x : x.split("-")[-1])
weather_1.drop(["YMD"],axis = 1,inplace=True)

In [24]:
weather_1

,STATION_NAME,HOURLYVISIBILITY,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPC,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYPrecip,TIME,Day
0,STL,10.0,27.8,24.5,23.3,77.0,6.0,NaN,00:51,01
1,STL,10.0,26.7,24.2,23.3,82.0,0.0,0.0,01:51,01
2,STL,10.0,25.6,23.8,23.3,87.0,5.0,0.0,02:51,01
3,STL,8.0,25.0,23.7,23.3,90.0,3.0,0.0,03:51,01
4,STL,9.0,25.6,23.8,23.3,87.0,0.0,0.0,04:51,01
...,...,...,...,...,...,...,...,...,...,...
39468,BNA,NaN,18.9,16.4,15.0,78.0,6.0,NaN,21:00,30
39469,BNA,10.0,18.9,16.2,14.4,75.0,0.0,0.0,21:53,30
39470,BNA,10.0,18.9,16.4,15.0,78.0,0.0,0.0,22:53,30
39471,BNA,10.0,18.9,16.4,15.0,78.0,3.0,0.0,23:53,30


In [25]:
weather_1.to_csv("C:/Users/USER/Desktop/t.csv",index= False)

# load data
weather_1 = pd.read_csv("C:/Users/USER/Desktop/t.csv")

# 由於兩個測站紀錄時間不同，因此分成兩個資料表處理
STL = weather_1[weather_1["STATION_NAME"].isin(["STL"])].reset_index(drop=True) # 就地重置索引
BNA = weather_1[weather_1["STATION_NAME"].isin(["BNA"])].reset_index(drop=True) # 就地重置索引

In [26]:
## 6. 由於STL測站天氣數據都是在每小時的51分紀錄，因此將不是51分的列刪除

# 新增"小時"欄位
STL["HOUR"] = STL["TIME"].apply(lambda x : int(x[:2]))

# 獨立"分"欄位，用來判斷此筆資料是否為51分的列
STL["min"] = STL["TIME"].apply(lambda x : int(x[-2:]))

# 將不是51分的資料抓出來
not_51 = STL[STL["min"] != 51]

# 刪除不是51分的列
STL = STL.drop(not_51.index)

# 將STL["min"]欄位刪除
STL.drop(["min"],axis=1,inplace = True)


## 6. 由於BNA測站天氣數據都是在每小時的53分紀錄，因此將不是53分的列刪除

# 新增"小時"欄位
BNA["HOUR"] = BNA["TIME"].apply(lambda x : int(x[:2]))

# 獨立"分"欄位，用來判斷此筆資料是否為53分的列
BNA["min"] = BNA["TIME"].apply(lambda x : int(x[-2:]))

# 將不是53分的資料抓出來
not_53 = BNA[BNA["min"] != 53]

# 刪除不是53分的列
BNA = BNA.drop(not_53.index)

# 將BNA["min"]欄位刪除
BNA.drop(["min"],axis=1,inplace = True)

In [27]:
STL.isnull().sum()

STATION_NAME               0
HOURLYVISIBILITY           0
HOURLYDRYBULBTEMPC         0
HOURLYWETBULBTEMPC         1
HOURLYDewPointTempC        0
HOURLYRelativeHumidity     0
HOURLYWindSpeed            0
HOURLYPrecip              36
TIME                       0
Day                        0
HOUR                       0
dtype: int64

In [28]:
BNA.isnull().sum()

STATION_NAME               0
HOURLYVISIBILITY           0
HOURLYDRYBULBTEMPC         0
HOURLYWETBULBTEMPC         0
HOURLYDewPointTempC        0
HOURLYRelativeHumidity     0
HOURLYWindSpeed            0
HOURLYPrecip              13
TIME                       0
Day                        0
HOUR                       0
dtype: int64

In [29]:
## 7. 空值處理

# 由於HOURLYPrecip的值大多為0，因此用0取代空值(眾數)
STL['HOURLYPrecip'].fillna(value=0,inplace=True)
BNA['HOURLYPrecip'].fillna(value=0,inplace=True)
# HOURLYWETBULBTEMPC只有一筆空值，用平均數取代
STL['HOURLYWETBULBTEMPC'] = STL['HOURLYWETBULBTEMPC'].fillna(STL['HOURLYWETBULBTEMPC'].mean())

In [30]:
## 8. 特徵組合:新增"蒸發量"欄位(乾球溫度與濕球溫度差距越大，表示蒸發量越大)

STL["evaporation"] = STL["HOURLYDRYBULBTEMPC"]-STL["HOURLYWETBULBTEMPC"]
BNA["evaporation"] = BNA["HOURLYDRYBULBTEMPC"]-BNA["HOURLYWETBULBTEMPC"]

#刪除乾溼球溫度欄位
STL.drop(["HOURLYDRYBULBTEMPC","HOURLYWETBULBTEMPC"],axis=1,inplace=True)
BNA.drop(["HOURLYDRYBULBTEMPC","HOURLYWETBULBTEMPC"],axis=1,inplace=True)

In [31]:
# 合併成一個檔案
weather_1 = pd.concat([STL, BNA],axis=0)

In [32]:
# 將清整後的檔案匯出
weather_1.to_csv("C:/Users/USER/Desktop/weather1.csv",index=False)

### 合併weather1&weather2

In [33]:
weather1 = pd.read_csv("C:/Users/USER/Desktop/weather1.csv")
weather2 = pd.read_csv("C:/Users/USER/Desktop/weather2.csv")
weather = pd.concat([weather1, weather2],axis=0)
print(weather.shape) # 合併後的csv有2880列 13欄

weather.to_csv("C:/Users/USER/Desktop/weather.csv",index=False)

(2880, 10)


In [34]:
weather

,STATION_NAME,HOURLYVISIBILITY,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYPrecip,TIME,Day,HOUR,evaporation
0,STL,10.0,23.3,77.0,6.0,0.0,00:51,1,0,3.3
1,STL,10.0,23.3,82.0,0.0,0.0,01:51,1,1,2.5
2,STL,10.0,23.3,87.0,5.0,0.0,02:51,1,2,1.8
3,STL,8.0,23.3,90.0,3.0,0.0,03:51,1,3,1.3
4,STL,9.0,23.3,87.0,0.0,0.0,04:51,1,4,1.8
...,...,...,...,...,...,...,...,...,...,...
1435,DAL,10.0,15.0,50.0,3.0,0.0,19:53,30,19,5.6
1436,DAL,10.0,16.1,58.0,6.0,0.0,20:53,30,20,4.9
1437,DAL,10.0,16.7,62.0,7.0,0.0,21:53,30,21,4.5
1438,DAL,10.0,16.7,62.0,7.0,0.0,22:53,30,22,4.5
